In [1]:
import pandas as pd, numpy as np

In [2]:
data=pd.read_csv("tic-tac-toe.data")
data

,x,x.1,x.2,x.3,o,o.1,x.4,o.2,o.3,positive
0,x,x,x,x,o,o,o,x,o,positive
1,x,x,x,x,o,o,o,o,x,positive
2,x,x,x,x,o,o,o,b,b,positive
3,x,x,x,x,o,o,b,o,b,positive
4,x,x,x,x,o,o,b,b,o,positive
...,...,...,...,...,...,...,...,...,...,...
952,o,x,x,x,o,o,o,x,x,negative
953,o,x,o,x,x,o,x,o,x,negative
954,o,x,o,x,o,x,x,o,x,negative
955,o,x,o,o,x,x,x,o,x,negative


In [3]:
data.shape

(957, 10)

In [4]:
def binarize(data):
    bind=pd.DataFrame()
    for i in data:
        if len(np.unique(data[i]))==2:
            bind[i]=np.zeros(data.shape[0])
            bind.loc[data[i]==np.unique(data[i])[1], [i]]=1
        elif len(np.unique(data[i]))>2:
#             for j in np.unique(data[i]):
#                 bind[i+"_"+j]=np.zeros(data.shape[0])
# #                 bind.loc[data[i]==j, [i+"_"+j]]=1
            bind[i+'x']=np.zeros(data.shape[0])
            bind.loc[data[i]=='x', [i+'x']]=1
#             bind[i]=data[i]
    return bind

In [5]:
beta=binarize(data)

In [6]:
beta

,xx,x.1x,x.2x,x.3x,ox,o.1x,x.4x,o.2x,o.3x,positive
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
952,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
953,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
954,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
955,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [7]:
win=beta["positive"]
win

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
952    0.0
953    0.0
954    0.0
955    0.0
956    0.0
Name: positive, Length: 957, dtype: float64

In [8]:
beta=beta.drop(["positive"], axis=1)

In [9]:
from sklearn.model_selection import train_test_split
test_size=0.1
X_train, X_test, y_train, y_test = train_test_split(beta, win, test_size=test_size)
X_train_pos=X_train.loc[y_train==1]
X_train_neg=X_train.loc[y_train==0]

y_train_pos=y_train.loc[y_train==1]
y_train_neg=y_train.loc[y_train==0]

X_test_pos=X_test.loc[y_test==1]
X_test_neg=X_test.loc[y_test==0]

y_test_pos=y_test.loc[y_test==1]
y_test_neg=y_test.loc[y_test==0]

In [10]:
def inter_bin(x, y):
    return x*y

In [11]:
def inter_cat(x,y):
    return [x, y]

In [12]:
def inter_num(x, y):
    return [np.min([x[0], y[0]]), np.max([x[1], y[1]])]

In [13]:
def inter(x,y, meta):
    res=[]
    for i in range(len(x)):
        if meta[i]=="bin":
            res.append(inter_bin(x[i], y[i]))
        elif meta[i]=="cat":
            res.append(inter_cat(x[i], y[i]))
        elif meta[i]=="num":
            res.append(inter_num(x[i], y[i]))
        else:
            raise Exception(str("Unknown metadata passed: "+meta[i]+" please use one of the following: bin, num, cat"))
    return np.array(res, dtype=object)

In [14]:
def isin(x, y, meta):
#     z=inter(x, y, meta)
#     print(z)
    res=True
    for i in range(len(x)):
        if meta[i]=="bin":
            res*=(y[i]>=x[i])
        elif meta[i]=="cat":
            res*=(y[i] in x[i])
        elif meta[i]=="num":
            res*=(x[i][0]<=y[i]<=x[i][1])
        else:
            raise Exception(str("Unknown metadata passed: "+meta[i]+" please use one of the following: bin, num, cat"))
        if not res: return res
    return res

In [15]:
def support(x, cl, meta):
    return np.mean([isin(x, i, meta) for i in cl])

In [16]:
def conf(x, y, a, b):
#     print(x, y)
    return np.mean([x[i]==y[i] for i in range(len(y))])

In [30]:
met=['bin' for i in range(data.shape[1])]
met

['bin', 'bin', 'bin', 'bin', 'bin', 'bin', 'bin', 'bin', 'bin', 'bin']

In [31]:
def classify(x, X_pos, X_neg, min_conf, minsup, aconf, meta, numdif=0, maxcont=0):
    cont_pos=0
    sup_pos=0
#     npos=0
#     minsuppos=np.inf
#     maxsuppos=0
#     mincontpos=np.inf
#     maxcontpos=0
    
    for i in X_pos:
        z=inter(i, x, meta)
        c=conf(i, x, meta, numdif)
#         print(c)
        if c>=min_conf:
#             print("c")
            return 1
           
        if c>=aconf:
#             print(c)
            sup=support(z, X_pos, meta)
            if sup>=minsup:
                return 1
            cont=support(z, X_neg, meta)
#             if cont<=maxcont:
#                 return 1
            cont_pos+=cont
            sup_pos+=sup
#             npos+=1
#             minsuppos=min(minsuppos, sup)
#             maxsuppos=max(maxsuppos, sup)
#             mincontpos=min(mincontpos, cont)
#             maxcontpos=max(maxcontpos, cont)
    #     print("+")
    
    cont_neg=0
    sup_neg=0
#     nneg=0
#     minsupneg=np.inf
#     maxsupneg=0
#     mincontneg=np.inf
#     maxcontneg=0
    
    for i in X_neg:
        z=inter(i, x, meta)
        c=conf(i, x, meta, numdif)
        if c>=min_conf:
            return 0
        
        if c>=aconf:
#             print(c)
            sup=support(z, X_neg, meta)
            if sup>=minsup:
                return 0
            cont=support(z, X_pos, meta)
#             if cont<=maxcont:
#                 return 0
            cont_neg+=cont
            sup_neg+=sup
#             nneg+=1
#             minsupneg=min(minsupneg, sup)
#             maxsupneg=max(maxsupneg, sup)
#             mincontneg=min(mincontneg, cont)
#             maxcontneg=max(maxcontneg, cont)      
#     print("-")
    
    
#     cont_pos/=npos
#     sup_pos/=npos
#     cont_neg/=nneg
#     sup_neg/=nneg
#     print(cont_pos,sup_pos, cont_neg,sup_neg)
#     print(minsupneg,
#             maxsupneg,
#             mincontneg,
#             maxcontneg,"\n" ,minsuppos,
#             maxsuppos,
#             mincontpos,
#             maxcontpos)
#     print(cont_pos,sup_pos, cont_neg,sup_neg)
    if sup_pos!=0 and sup_neg!=0:
        return int(cont_pos/sup_pos<cont_neg/sup_neg)
    elif sup_pos==0 or sup_neg==0 and not sup_pos==sup_neg:
        return int(sup_pos/X_pos.shape[0]>sup_neg/X_neg.shape[0])
    else: return int(cont_pos/X_pos.shape[0]<cont_neg/X_neg.shape[0])
#     return int(maxcontpos<maxcontneg)
#     return int(maxsuppos>maxsupneg)

In [41]:
%time classify(X_test_pos.loc[X_test_pos.index[1]].values, X_train_pos.values, X_train_neg.values, 0.9, 0.9, 0.7, met)

Wall time: 213 ms


1

In [42]:
%time classify(X_test_neg.loc[X_test_neg.index[1]].values, X_train_pos.values, X_train_neg.values, 0.9, 0.9, 0.7, met)

Wall time: 870 ms


0

In [43]:
%time y=np.array([classify(X_test_pos.loc[i].values, X_train_pos.values, X_train_neg.values, 0.9, 0.9, 0.7, met) for i in X_test_pos.index])

Wall time: 32.9 s


In [44]:
%time yn=np.array([classify(X_test_neg.loc[i].values, X_train_pos.values, X_train_neg.values, 0.9, 0.9, 0.7, met) for i in X_test_neg.index])

Wall time: 36.1 s


In [45]:
def metrics(y, yn, y_true, yn_true):
    m=dict()
    FN=sum(y^y_true)
    FP=sum(yn^yn_true)
    TP=len(y_true)-FN
    TN=len(yn_true)-FP
    m["FN"]=FN
    m["FP"]=FP
    m["TP"]=TP
    m["TN"]=TN
    m["A"]=(TP+TN)/(FP+FN+TP+TN)
    m["TPR"]=TP/(TP+FN)
    m["TNR"]=TN/(TN+FP)
    m["NPV"]=TN/(TN+FN)
    m["PPV"]=TP/(TP+FP)
    m["FPR"]=FP/(FP+TN)
    m["FDR"]=FP/(FP+TP)
    m["P"]=1/2*(TP/(TP+FP)+TN/(TN+FN))
    m["R"]=1/2*(TP/(TP+FN)+TN/(TN+FP))
    return m

True Positive
True Negative
False Positive
False Negative
True Positive Rate
True Negative Rate
Negative Predictive Value
False Positive Rate
False Discovery Rate
Accuracy
Precision
Recall

In [46]:
metrics(y, yn, np.array(y_test_pos, dtype=int), np.array(y_test_neg,  dtype=int))

{'FN': 0,
 'FP': 3,
 'TP': 63,
 'TN': 31,
 'A': 0.9690721649484536,
 'TPR': 1.0,
 'TNR': 0.9117647058823529,
 'NPV': 1.0,
 'PPV': 0.9545454545454546,
 'FPR': 0.08823529411764706,
 'FDR': 0.045454545454545456,
 'P': 0.9772727272727273,
 'R': 0.9558823529411764}

In [47]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True)

In [48]:
X_pos=beta.loc[win==1]
X_neg=beta.loc[win==0]
X_pos.index=[i for i in range(X_pos.shape[0])]
X_neg.index=[i for i in range(X_neg.shape[0])]
y_pos=win.loc[win==1]
y_neg=win.loc[win==0]
y_pos.index=[i for i in range(X_pos.shape[0])]
y_neg.index=[i for i in range(y_neg.shape[0])]
l=[i for i in kf.split(X_pos)]
k=[i for i in kf.split(X_neg)]
a=np.zeros(13)
for i in range(10):

        print(i)
        X_train_pos=X_pos.loc[l[i][0]]
        X_train_neg=X_neg.loc[k[i][0]]

        y_train_pos=y_pos.loc[l[i][0]]
        y_train_neg=y_neg.loc[k[i][0]]

        X_test_pos=X_pos.loc[l[i][1]]
        X_test_neg=X_neg.loc[k[i][1]]

        y_test_pos=y_pos.loc[l[i][1]]
        y_test_neg=y_neg.loc[k[i][1]]

        y=np.array([classify(X_test_pos.loc[i].values, X_train_pos.values, X_train_neg.values, 0.9, 0.9, 0.7, met) for i in X_test_pos.index])
        yn=np.array([classify(X_test_neg.loc[i].values, X_train_pos.values, X_train_neg.values, 0.9, 0.9, 0.7, met) for i in X_test_neg.index])
        a+=list(metrics(y, yn, np.array(y_test_pos, dtype=int), np.array(y_test_neg,  dtype=int)).values())
a/10

0
1
2
3
4
5
6
7
8
9


array([0.00000000e+00, 3.30000000e+00, 6.25000000e+01, 2.99000000e+01,
       9.65546437e-01, 1.00000000e+00, 9.00713012e-01, 1.00000000e+00,
       9.50599757e-01, 9.92869875e-02, 4.94002427e-02, 9.75299879e-01,
       9.50356506e-01])

In [49]:
param_grid={
    "minconf":[0.8,0.9],
    "minsup":[0.8,0.9],
    "aconf":[0.5,0.7]
}

In [51]:
pg=[(i, j, k) for i in param_grid["minconf"] for j in param_grid["minsup"]  for k in param_grid["aconf"]]
pg

[(0.8, 0.8, 0.5),
 (0.8, 0.8, 0.7),
 (0.8, 0.9, 0.5),
 (0.8, 0.9, 0.7),
 (0.9, 0.8, 0.5),
 (0.9, 0.8, 0.7),
 (0.9, 0.9, 0.5),
 (0.9, 0.9, 0.7)]

In [52]:
res=pd.DataFrame(columns=["minconf", "minsup", "aconf", "FN", "FP","TP","TN","A","TPR","TNR","NPV","PPV","FPR","FDR","P","R"])
res

,minconf,minsup,aconf,FN,FP,TP,TN,A,TPR,TNR,NPV,PPV,FPR,FDR,P,R


In [53]:
X_pos=beta.loc[win==1]
X_neg=beta.loc[win==0]
X_pos.index=[i for i in range(X_pos.shape[0])]
X_neg.index=[i for i in range(X_neg.shape[0])]
y_pos=win.loc[win==1]
y_neg=win.loc[win==0]
y_pos.index=[i for i in range(X_pos.shape[0])]
y_neg.index=[i for i in range(y_neg.shape[0])]
l=[i for i in kf.split(X_pos)]
k=[i for i in kf.split(X_neg)]

In [ ]:
d=0
for j in pg:
    a=np.zeros(13)
    for i in range(10):

        X_train_pos=X_pos.loc[l[i][0]]
        X_train_neg=X_neg.loc[k[i][0]]

        y_train_pos=y_pos.loc[l[i][0]]
        y_train_neg=y_neg.loc[k[i][0]]

        X_test_pos=X_pos.loc[l[i][1]]
        X_test_neg=X_neg.loc[k[i][1]]

        y_test_pos=y_pos.loc[l[i][1]]
        y_test_neg=y_neg.loc[k[i][1]]

        y=np.array([classify(X_test_pos.loc[i].values, X_train_pos.values, X_train_neg.values, j[0], j[1], j[2], met) for i in X_test_pos.index])
        yn=np.array([classify(X_test_neg.loc[i].values, X_train_pos.values, X_train_neg.values, j[0], j[1], j[2], met) for i in X_test_neg.index])
        a+=list(metrics(y, yn, np.array(y_test_pos, dtype=int), np.array(y_test_neg,  dtype=int)).values())
    print(j)
    a/=10
    res.loc[d]=list(j)+list(a)
    d+=1

(0.8, 0.8, 0.5)
(0.8, 0.8, 0.7)
(0.8, 0.9, 0.5)
(0.8, 0.9, 0.7)


In [ ]:
res.to_csv("ttt_bin.csv")

In [ ]:
res